    16_230109_100_kyungki_apt_부천

     
     data : /aiffel/aiffel/0000_AIFFELTHON/3김연수/002_data_dir/002_apt_data/apt_부천_521.xls

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [3]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [4]:
#9 Folium를 활용한 공간 데이터 시각화
### 전국도시철도역사정보표준데이터 가져오기

apt_df = pd.read_excel("/aiffel/aiffel/0000_AIFFELTHON/3김연수/002_data_dir/002_apt_data/apt_부천_521.xls")
apt_df.head()

,시군명,공동주택구분,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,부천시,NaN,(건우3차아파트),송내동,328-1,1986-06-14,5,1,124,N
1,부천시,NaN,(국경아파트),괴안동,26696,1988-02-12,5,1,45,N
2,부천시,NaN,(근영아파트),송내동,593-5,1982-10-27,4,1,20,N
3,부천시,NaN,(대보아파트),송내동,336-7,1988-03-28,5,1,30,N
4,부천시,NaN,(대성아파트),괴안동,120-12,1986-12-29,5,1,30,N


In [5]:
apt_df.drop(["공동주택구분"], axis=1,inplace=True)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,부천시,(건우3차아파트),송내동,328-1,1986-06-14,5,1,124,N
1,부천시,(국경아파트),괴안동,26696,1988-02-12,5,1,45,N
2,부천시,(근영아파트),송내동,593-5,1982-10-27,4,1,20,N
3,부천시,(대보아파트),송내동,336-7,1988-03-28,5,1,30,N
4,부천시,(대성아파트),괴안동,120-12,1986-12-29,5,1,30,N


In [6]:
apt_df['지번주소'] = apt_df['지번주소'].astype(str)

In [7]:
#okok
apt_df['addr'] = '경기도 ' + apt_df[['시군명', '읍면동주소','지번주소']].agg(' '.join, axis=1)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr
0,부천시,(건우3차아파트),송내동,328-1,1986-06-14,5,1,124,N,경기도 부천시 송내동 328-1
1,부천시,(국경아파트),괴안동,26696,1988-02-12,5,1,45,N,경기도 부천시 괴안동 26696
2,부천시,(근영아파트),송내동,593-5,1982-10-27,4,1,20,N,경기도 부천시 송내동 593-5
3,부천시,(대보아파트),송내동,336-7,1988-03-28,5,1,30,N,경기도 부천시 송내동 336-7
4,부천시,(대성아파트),괴안동,120-12,1986-12-29,5,1,30,N,경기도 부천시 괴안동 120-12


In [8]:
# https://sgisapi.kostat.go.kr/OpenAPI3/addr/stage.json

In [9]:
#5 seoul university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [10]:
#5 seoul university .. 주소 기반 위도,경도 추가
cnt = 0
for _, row in apt_df.iterrows():
    try:
        x, y = get_adress_x_y(row["addr"])
     
        apt_df.loc[_, "x"] = x
        apt_df.loc[_, "y"] = y
    except:
        print(row["addr"] + '은(는) 검색에 실패했습니다')
        cnt = cnt + 1

print("실패건수 : ", cnt)
apt_df.head()


경기도 부천시 괴안동 26696은(는) 검색에 실패했습니다
경기도 부천시 괴안동 44645은(는) 검색에 실패했습니다
경기도 부천시 심곡동 13455은(는) 검색에 실패했습니다
경기도 부천시 심곡동 34700은(는) 검색에 실패했습니다
경기도 부천시 작동 27426은(는) 검색에 실패했습니다
경기도 부천시 괴안동 32599은(는) 검색에 실패했습니다
경기도 부천시 여월동 19664은(는) 검색에 실패했습니다
경기도 부천시 도당동 18354은(는) 검색에 실패했습니다
경기도 부천시 작동 25934은(는) 검색에 실패했습니다
경기도 부천시 소사본동 20149은(는) 검색에 실패했습니다
경기도 부천시 소사본동 20121은(는) 검색에 실패했습니다
경기도 부천시 역곡동 44583은(는) 검색에 실패했습니다
경기도 부천시 원종동 24593은(는) 검색에 실패했습니다
경기도 부천시 도당동 16803은(는) 검색에 실패했습니다
경기도 부천시 소사동 44618은(는) 검색에 실패했습니다
경기도 부천시 원미동 21186은(는) 검색에 실패했습니다
경기도 부천시 괴안동 14은(는) 검색에 실패했습니다
경기도 부천시 역곡동 30713은(는) 검색에 실패했습니다
경기도 부천시 원종동 25569은(는) 검색에 실패했습니다
경기도 부천시 도당동 25993은(는) 검색에 실패했습니다
경기도 부천시 원종동 44588은(는) 검색에 실패했습니다
경기도 부천시 역곡동 44707은(는) 검색에 실패했습니다
경기도 부천시 괴안동 44758은(는) 검색에 실패했습니다
경기도 부천시 작동 44682은(는) 검색에 실패했습니다
경기도 부천시 심곡동 13150은(는) 검색에 실패했습니다
경기도 부천시 내동 19815은(는) 검색에 실패했습니다
경기도 부천시 도당동 33604은(는) 검색에 실패했습니다
경기도 부천시 괴안동 36161은(는) 검색에 실패했습니다
경기도 부천시 역곡동 27061은(는) 검색에 실패했습니다
경기도 부천시 역곡동 51-18은(는) 검색에 실패했습니다
경기도 부천시 역곡동 51-

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y
0,부천시,(건우3차아파트),송내동,328-1,1986-06-14,5,1,124,N,경기도 부천시 송내동 328-1,934212.629401194,1943056.78576311
1,부천시,(국경아파트),괴안동,26696,1988-02-12,5,1,45,N,경기도 부천시 괴안동 26696,NaN,NaN
2,부천시,(근영아파트),송내동,593-5,1982-10-27,4,1,20,N,경기도 부천시 송내동 593-5,935635.583207981,1942717.13178349
3,부천시,(대보아파트),송내동,336-7,1988-03-28,5,1,30,N,경기도 부천시 송내동 336-7,934381.64770609,1943314.89730381
4,부천시,(대성아파트),괴안동,120-12,1986-12-29,5,1,30,N,경기도 부천시 괴안동 120-12,939357.749350224,1942593.17743961


In [11]:
#5 seoul university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [12]:
#5 seoul university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in apt_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        apt_df.loc[_, "lng"] = x
        apt_df.loc[_, "lat"] = y

    except:
        pass
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y,lng,lat
0,부천시,(건우3차아파트),송내동,328-1,1986-06-14,5,1,124,N,경기도 부천시 송내동 328-1,934212.629401194,1943056.78576311,126.755876,37.484411
1,부천시,(국경아파트),괴안동,26696,1988-02-12,5,1,45,N,경기도 부천시 괴안동 26696,NaN,NaN,NaN,NaN
2,부천시,(근영아파트),송내동,593-5,1982-10-27,4,1,20,N,경기도 부천시 송내동 593-5,935635.583207981,1942717.13178349,126.771999,37.481450
3,부천시,(대보아파트),송내동,336-7,1988-03-28,5,1,30,N,경기도 부천시 송내동 336-7,934381.64770609,1943314.89730381,126.757764,37.486749
4,부천시,(대성아파트),괴안동,120-12,1986-12-29,5,1,30,N,경기도 부천시 괴안동 120-12,939357.749350224,1942593.17743961,126.814107,37.480584


In [13]:
#5 seoul university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [14]:
# #5 seoul university # 대학 이름, 위도, 경도를 가져와 시각화
# for _, row in apt_df.iterrows():
#     try:
#         folium.Marker(
#             location=[row["lat"], row["lng"]], # 좌표
#             popup=row["공동주택명정보"], # 클릭시 표시될 popup 내용
#             icon=folium.Icon(color="cadetblue", icon="info-sign") # 표시될 아이콘 설정
#         ).add_to(map)
#     except:
#         pass

# print('슝~')

In [15]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in apt_df.iterrows():
    try:
        folium.Marker(
            location=[apt_df.loc[idx, "lat"], apt_df.loc[idx, "lng"]],
            popup=apt_df.loc[idx, "공동주택명정보"],
            icon=folium.Icon(color="cadetblue", icon="house")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [16]:
#5 seoul university 
map

In [17]:
map.save("/aiffel/aiffel/0000_AIFFELTHON/3김연수/003_map_xml/apt_buchon.html")

In [18]:
apt_df.to_csv("apt_buchon.csv", index = False)